In [1]:
# Import the necessary packages
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
# Load dataset
data = pd.read_csv('source/data.csv')

In [3]:
data.head()

,AccID,day,month,year,time,lum,atm_condition,collision_type,lat,long,...,manv,motor,seat,user_category,gravity,gender,birth_year,reason_travel,safety_equipment1,age
0,201900000001,30,11,2019,5400000,4,1,2,48.896210,2.470120,...,23,1,2,2,4,2,2002.0,0,1,17.0
1,201900000001,30,11,2019,5400000,4,1,2,48.896210,2.470120,...,23,1,1,1,4,2,1993.0,5,1,26.0
2,201900000001,30,11,2019,5400000,4,1,2,48.896210,2.470120,...,11,1,1,1,1,1,1959.0,0,1,60.0
3,201900000002,30,11,2019,10200000,3,1,6,48.930700,2.368800,...,0,1,1,1,4,2,1994.0,0,1,25.0
4,201900000003,28,11,2019,54900000,1,1,4,48.935872,2.319174,...,2,1,1,1,1,1,1996.0,0,1,23.0


In [4]:
# Selecting features and target variable
features = ['lum','atm_condition','collision_type','route_category','traffic_regime','reserved_lane_code','longitudinal_profile','upstream_terminal_number','plan','surface_condition','infra','accident_situation','traffic_direction','vehicle_category','fixed_obstacle','mobile_obstacle',
'initial_impact_point','manv','motor','seat','user_category','gender','reason_travel','safety_equipment1','maximum_speed','age','lat','long','distance_upstream_terminal','total_number_lanes','day','month','year','time']
target = 'gravity'

In [5]:
# Handling categorical features with label encoding
le = LabelEncoder()

In [6]:
# Encoding categorical variables
for col in ['lum','atm_condition','collision_type','route_category','traffic_regime','reserved_lane_code','longitudinal_profile','upstream_terminal_number','plan','surface_condition','infra','accident_situation','traffic_direction','vehicle_category','fixed_obstacle','mobile_obstacle',
'initial_impact_point','manv','motor','seat','user_category','gender','reason_travel','safety_equipment1']:
    data[col] = le.fit_transform(data[col])

In [7]:
# Splitting the data into train and test sets
X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

C:\Users\sd10725\AppData\Roaming\Python\Python311\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\sd10725\AppData\Roaming\Python\Python311\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [9]:
# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300],           # Number of trees
    'max_depth': [10, 20, 30],                 # Maximum depth of trees
    'min_samples_split': [2, 5, 10],           # Minimum samples required to split
    'min_samples_leaf': [1, 2, 4],             # Minimum samples in leaf nodes
    'max_features': ['auto', 'sqrt'],          # Number of features to consider at each split
    'bootstrap': [True, False],                # Whether bootstrap samples are used
    'class_weight': [{1: 1, 2: 20, 3: 1, 4: 1}] # Increased weight for fatalities
}

In [10]:
# Initialize the Logistic Regression model
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)

In [11]:
# Train the model
log_reg_model.fit(X_res, y_res)

LogisticRegression(max_iter=1000, random_state=42)

In [12]:
# Predict on the test set
y_pred_log_reg = log_reg_model.predict(X_test)

In [13]:
# Evaluate the model
conf_matrix_log_reg = confusion_matrix(y_test, y_pred_log_reg)
class_report_log_reg = classification_report(y_test, y_pred_log_reg)

C:\Users\sd10725\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sd10725\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sd10725\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [14]:
conf_matrix_log_reg

array([[37314,    57,     0,     0],
       [ 2326,     9,     0,     0],
       [13705,    32,     0,     0],
       [36014,    77,     0,     0]], dtype=int64)

In [15]:
print(class_report_log_reg)

              precision    recall  f1-score   support

           1       0.42      1.00      0.59     37371
           2       0.05      0.00      0.01      2335
           3       0.00      0.00      0.00     13737
           4       0.00      0.00      0.00     36091

    accuracy                           0.42     89534
   macro avg       0.12      0.25      0.15     89534
weighted avg       0.18      0.42      0.25     89534

